In [1]:
import pandas as pd

In [2]:
df = pd.read_json("../data/text-davinci-003-single-response.json") \
       .sort_values(["question", "temperature", "frequency_penalty"]) \
       .reset_index(drop=True)
df.head()

,question,response,model,max_tokens,stop,prompt,temperature,frequency_penalty
0,How do birthday cakes make you feel?,Birthday cakes make me feel both happy and sad...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
1,How do birthday cakes make you feel?,Birthday cakes make me feel nostalgic. I alway...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
2,How do birthday cakes make you feel?,Birthdays cakes make me feel excited and happy...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.01
3,How do birthday cakes make you feel?,Birthday cakes make me feel nostalgic and happ...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.10
4,How do birthday cakes make you feel?,Birthday cakes make me feel really happy and e...,text-davinci-003,128,"[Therapist:, Patient:]",Below is a conversation between a patient and ...,0.7,0.10


In [3]:
df.to_excel("../data/text-davinci-003-single-response.xlsx", index=False)